In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [2]:
import os

os.environ["GOOGLE_API_KEY"] = '******'


Get data and rename columns

In [3]:
request = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
my_table  = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M',header=0)[0]
my_table.rename(columns = {'Postcode':'PostalCode'},inplace=True)

Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [4]:
my_table = my_table.set_index('Borough').drop('Not assigned').reset_index()

In [5]:
my_table.head()

,Borough,PostalCode,Neighbourhood
0,North York,M3A,Parkwoods
1,North York,M4A,Victoria Village
2,Downtown Toronto,M5A,Harbourfront
3,Downtown Toronto,M5A,Regent Park
4,North York,M6A,Lawrence Heights


More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

In [6]:
my_table = my_table.groupby(['PostalCode','Borough'])['Neighbourhood'].apply(', '.join).reset_index()

If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.

In [7]:
my_table.loc[my_table['Neighbourhood']=='Not assigned','Neighbourhood'] = my_table.loc[my_table['Neighbourhood']=='Not assigned']['Borough'] 

Verifying the above action

In [8]:
my_table.loc[my_table['PostalCode']=='M5A']

,PostalCode,Borough,Neighbourhood
53,M5A,Downtown Toronto,"Harbourfront, Regent Park"


In [9]:
my_table.loc[my_table['Neighbourhood']=="Queen's Park"]

,PostalCode,Borough,Neighbourhood
85,M7A,Queen's Park,Queen's Park


All Done!

In [10]:
my_table.shape

(103, 3)

In [11]:
!pip install geocoder

In [12]:

import geocoder

In [13]:

latitude =[]
longitude=[]
for postal_code in my_table['PostalCode']:
    #postal_code = 'M5G'
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
      
      g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))

      lat_lng_coords = g.latlng
    
    latitude.append(lat_lng_coords[0])
    longitude.append(lat_lng_coords[1])

In [14]:
my_table['Latitude']=latitude

In [15]:
my_table['Longitude']=longitude

In [16]:
my_table.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
